In [2]:
import praw

# Configuration de l'API Reddit
reddit = praw.Reddit(client_id='9seiF8n3LPJv4EDar1GyuA',
                     client_secret='LawYhA2r4-fQTWEi2yYT4YXID5V2sQ',
                     user_agent='YOUR_USER_AGENT')

# Fonction pour collecter les données d'un subreddit
def collect_reddit_data(subreddit_name, limit=10):
    subreddit = reddit.subreddit(subreddit_name)
    posts = []
    for post in subreddit.hot(limit=limit):
        posts.append({
            'title': post.title,
            'score': post.score,
            'id': post.id,
            'url': post.url,
            'num_comments': post.num_comments,
            'created': post.created,
            'body': post.selftext
        })
    return posts

# Exemple d'utilisation
subreddit_name = 'travel'
data = collect_reddit_data(subreddit_name, limit=5)
for post in data:
    print(post)

{'title': 'All Layover Questions - READ THIS NOTICE', 'score': 144, 'id': '1dzc3zh', 'url': 'https://www.reddit.com/r/travel/comments/1dzc3zh/all_layover_questions_read_this_notice/', 'num_comments': 46, 'created': 1720555637.0, 'body': '**READ THE NEW LAYOVER FAQ:** [**https://www.reddit.com/r/travel/wiki/mfaq-flying/layovers**](https://www.reddit.com/r/travel/wiki/mfaq-flying/layovers)\n\nAll layover questions will be removed unless your situation is unique and cannot be answered by the wiki.\n\n**Members of the community**: please report any layover questions that can be answered by the wiki and we will remove them promptly.\n\nSelf-transfers times are not covered under this new guideline and wiki.'}
{'title': 'Serious question: is there a worse airport than CDG?', 'score': 108, 'id': '1i3df08', 'url': 'https://www.reddit.com/r/travel/comments/1i3df08/serious_question_is_there_a_worse_airport_than_cdg/', 'num_comments': 280, 'created': 1737109594.0, 'body': 'I’m traditionally kind o

In [3]:
data

[{'title': 'All Layover Questions - READ THIS NOTICE',
  'score': 144,
  'id': '1dzc3zh',
  'url': 'https://www.reddit.com/r/travel/comments/1dzc3zh/all_layover_questions_read_this_notice/',
  'num_comments': 46,
  'created': 1720555637.0,
  'body': '**READ THE NEW LAYOVER FAQ:** [**https://www.reddit.com/r/travel/wiki/mfaq-flying/layovers**](https://www.reddit.com/r/travel/wiki/mfaq-flying/layovers)\n\nAll layover questions will be removed unless your situation is unique and cannot be answered by the wiki.\n\n**Members of the community**: please report any layover questions that can be answered by the wiki and we will remove them promptly.\n\nSelf-transfers times are not covered under this new guideline and wiki.'},
 {'title': 'Serious question: is there a worse airport than CDG?',
  'score': 108,
  'id': '1i3df08',
  'url': 'https://www.reddit.com/r/travel/comments/1i3df08/serious_question_is_there_a_worse_airport_than_cdg/',
  'num_comments': 280,
  'created': 1737109594.0,
  'body'

In [8]:
import openai
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('interesting_reddit_posts.csv')

# Function to analyze each post using OpenAI GPT
def analyze_post(title, body):
    if not title and not body:
        return {"Country": "/", "Satisfaction Score": "/", "Security Score": "/"}
    
    prompt = f"""Title: {title}\n\nBody: {body}\n\nAnalyze the Reddit post below and extract the following information:

Satisfaction Score: Rate the overall satisfaction described in the post between 0 and 10. If no satisfaction level can be inferred, return /.
Country: Identify the country mentioned in the post. If no country is mentioned, return /.
Security Score: Rate the security level described in the post between 0 and 10 based on the user's impressions. If the post does not mention security, return /.
Output format:
Satisfaction Score: [x/10 or /]
Country: [Country or /]
Security Score: [x/10 or /]"""
    
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=100
        )
        # Parse the response
        content = response.choices[0].message['content'].strip()
        # Extract fields from the response
        result = {
            "Country": extract_field(content, "Country"),
            "Satisfaction Score": extract_field(content, "Satisfaction Score"),
            "Security Score": extract_field(content, "Security Score")
        }
        return result
    except Exception as e:
        print(f"Error analyzing post: {e}")
        return {"Country": "/", "Satisfaction Score": "/", "Security Score": "/"}

# Helper function to extract a specific field from the response
def extract_field(text, field_name):
    import re
    match = re.search(rf"{field_name}: (.+?)(?:\n|$)", text)
    return match.group(1).strip() if match else "/"

# Analyze posts and add results to the DataFrame
df[['Country', 'Satisfaction Score', 'Security Score']] = df.apply(
    lambda row: pd.Series(analyze_post(row.get('title', ''), row.get('body', ''))),
    axis=1
)

# Save the updated DataFrame to a new CSV file
output_file = 'analyzed_posts2.csv'
df.to_csv(output_file, index=False)

print(f"The analysis has been completed and saved to '{output_file}'.")


The analysis has been completed and saved to 'analyzed_posts2.csv'.
